In [ ]:

!time pip install rdkit-pypi

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/

real	0m3.071s
user	0m2.732s
sys	0m0.305s


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
# import seaborn as sn
# import scipy as sp

import random
import tensorflow
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dense, BatchNormalization, Conv2D, Dropout, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier, KerasRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold

from rdkit import Chem # A core definition
from rdkit.Chem.Draw import MolsToGridImage # For displaying multiple molecules
from rdkit.Chem.AllChem import * # conformer generation and adding H
from rdkit.Chem.rdMolDescriptors import * # To calculate descriptors
from rdkit.Chem.Draw import IPythonConsole # This displays the molecule in-line
from rdkit.Chem import PandasTools # for pandas dataframe with rdkit
from rdkit.DataStructs.cDataStructs import ConvertToNumpyArray
import pandas.io.formats.style

# Dual active models

In [ ]:
import pandas as pd
sxl= pd.read_csv("M_sexual_LibrariesC_hits_nd_2ndclst_nhits.csv")
sxl['S_Hit'] = sxl['S_Hit'].replace(np.nan, "No")
sxl = sxl.iloc[: , 1:]
print(sxl['S_Hit'].value_counts())
sxl

## MACCSKeysFingerprint

In [ ]:
!pip install deepchem

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 693.2/693.2 KB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 MB 18.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.4/29.4 MB 65.7 MB/s eta 0:00:00
  Attempting uninstall: scipy
    Found existing installation: scipy 1.10.1
    Uninstalling scipy-1.10.1:
      Successfully uninstalled scipy-1.10.1


In [ ]:
import deepchem as dc
from deepchem.feat import MACCSKeysFingerprint
sxl["Target"] = np.where(sxl["S_Hit"]== "Yes", 1,0)
sxl["Target"].value_counts()

0    8975
1     916
Name: Target, dtype: int64

0       NC(=O)c1cccc2nc([C@H]3CCN(Cc4ccccn4)C3)[nH]c12
1                Fc1ccc2[nH]c(C3CCCN3Cc3c[nH]cn3)nc2c1
2                        COc1c(C)cnc(CN(C)C2CCCCC2)c1C
3                Cc1cccc(CN2CCCC2c2cccc(Nc3nccs3)n2)c1
4                   CCSCCNc1nc(-c2ccccn2)nc2c1c(C)nn2C
                             ...                      
9886          Cc1nc(C)n(C2CCCN(C(=O)c3ncoc3C(C)C)C2)n1
9887              COCCN1CCN(Cc2c(C)noc2C)Cc2ccc(C)nc21
9888               CCc1ncnc2c1CCN(Cc1ccc3c(c1)OCO3)CC2
9889              CCC1CN(CC(=O)NCCN2CCCCCC2)c2ccccc2S1
9890                Cc1cnn(CC2CCCN2Cc2nc3ccccc3nc2C)c1
Name: Smiles, Length: 9891, dtype: object

In [ ]:
dchm= sxl["Smiles"]
mols = [Chem.MolFromSmiles(x) for x in dchm]
featurizer = MACCSKeysFingerprint()
maccs = featurizer.featurize(mols)
df=[]
df = pd.DataFrame(maccs)
#df["Target"]= sxl["Target"]
df

array([[0, 0, 0, ..., 1, 1, 0],
       [0, 0, 0, ..., 0, 1, 0],
       [0, 0, 0, ..., 1, 1, 0],
       ...,
       [0, 0, 0, ..., 1, 1, 0],
       [0, 0, 0, ..., 1, 1, 0],
       [0, 0, 0, ..., 0, 1, 0]])

In [ ]:
from sklearn.model_selection import train_test_split
Xt_train, Xt_test, Yt_train, Yt_test = train_test_split(df, sxl["Target"],test_size=0.2, random_state=42)

# Hyperparameter tuning


---



---



In [ ]:
## hyper-parameters (like what C or gamma values to use)
# load library
from sklearn.svm import SVC

# we can add class_weight='balanced' to add panalize mistake
svc_model = SVC(class_weight='balanced', probability=True)

In [ ]:
kernels = ['Polynomial', 'RBF', 'Sigmoid','Linear']#A function which returns the corresponding SVC model
def getClassifier(ktype):
    if ktype == 0:
        # Polynomial kernal
        return SVC(kernel='poly', degree=8, gamma="auto",class_weight='balanced', probability=True)
    elif ktype == 1:
        # Radial Basis Function kernal
        return SVC(kernel='rbf', gamma="auto", class_weight='balanced', probability=True)
    elif ktype == 2:
        # Sigmoid kernal
        return SVC(kernel='sigmoid', gamma="auto", class_weight='balanced', probability=True)
    elif ktype == 3:
        # Linear kernal
        return SVC(kernel='linear', gamma="auto", class_weight='balanced', probability=True)

In [ ]:
## https://www.vebuso.com/2020/03/svm-hyperparameter-tuning-using-gridsearchcv/
import pandas as pd
import numpy as np
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import recall_score, precision_recall_curve,precision_score, confusion_matrix, accuracy_score, f1_score, roc_auc_score
import matplotlib.pyplot as plt
%matplotlib inline

for i in range(4):
    # Separate data into test and training sets
    X_train, X_test, y_train, y_test = train_test_split(df, sxl["Target"],test_size=0.2, random_state=42)# Train a SVC model using different kernal
    svclassifier = getClassifier(i)
    svclassifier.fit(X_train, y_train)# Make prediction
    y_pred = svclassifier.predict(X_test)# Evaluate our model
    print("Evaluation:", kernels[i], "kernel")
    print(classification_report(y_test,y_pred))

Evaluation: Polynomial kernel
              precision    recall  f1-score   support

           0       0.00      0.00      0.00      1788
           1       0.10      1.00      0.18       191

    accuracy                           0.10      1979
   macro avg       0.05      0.50      0.09      1979
weighted avg       0.01      0.10      0.02      1979



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Evaluation: RBF kernel
              precision    recall  f1-score   support

           0       0.99      0.93      0.96      1788
           1       0.56      0.88      0.68       191

    accuracy                           0.92      1979
   macro avg       0.77      0.90      0.82      1979
weighted avg       0.95      0.92      0.93      1979

Evaluation: Sigmoid kernel
              precision    recall  f1-score   support

           0       0.98      0.90      0.94      1788
           1       0.48      0.87      0.62       191

    accuracy                           0.90      1979
   macro avg       0.73      0.89      0.78      1979
weighted avg       0.94      0.90      0.91      1979

Evaluation: Linear kernel
              precision    recall  f1-score   support

           0       0.98      0.92      0.95      1788
           1       0.54      0.86      0.67       191

    accuracy                           0.92      1979
   macro avg       0.76      0.89      0.81      197

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold

# defining parameter range
param_grid = {'C': [0.1, 1, 10, 100, 1000],
              'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
              'kernel': ['rbf']}

grid = GridSearchCV(SVC(class_weight='balanced', probability=True), param_grid,cv=KFold(n_splits=10), refit = True, verbose = 3)

# fitting the model for grid search
grid.fit(Xt_train, Yt_train)

Fitting 10 folds for each of 25 candidates, totalling 250 fits
[CV 1/10] END .......C=0.1, gamma=1, kernel=rbf;, score=0.894 total time=  54.0s
[CV 2/10] END .......C=0.1, gamma=1, kernel=rbf;, score=0.915 total time=  44.9s
[CV 3/10] END .......C=0.1, gamma=1, kernel=rbf;, score=0.896 total time=  41.0s
[CV 4/10] END .......C=0.1, gamma=1, kernel=rbf;, score=0.914 total time=  42.9s
[CV 5/10] END .......C=0.1, gamma=1, kernel=rbf;, score=0.913 total time=  49.0s
[CV 6/10] END .......C=0.1, gamma=1, kernel=rbf;, score=0.932 total time=  48.4s
[CV 7/10] END .......C=0.1, gamma=1, kernel=rbf;, score=0.930 total time=  47.9s
[CV 8/10] END .......C=0.1, gamma=1, kernel=rbf;, score=0.917 total time=  43.7s
[CV 9/10] END .......C=0.1, gamma=1, kernel=rbf;, score=0.913 total time=  43.0s
[CV 10/10] END ......C=0.1, gamma=1, kernel=rbf;, score=0.919 total time=  41.9s
[CV 1/10] END .....C=0.1, gamma=0.1, kernel=rbf;, score=0.933 total time=  23.6s
[CV 2/10] END .....C=0.1, gamma=0.1, kernel=rb

In [ ]:

grid_predictions = grid.predict(Xt_test)

# print classification report
print(classification_report(Yt_test, grid_predictions))

              precision    recall  f1-score   support

           0       0.98      0.99      0.98      1788
           1       0.86      0.80      0.83       191

    accuracy                           0.97      1979
   macro avg       0.92      0.89      0.91      1979
weighted avg       0.97      0.97      0.97      1979



In [ ]:
print(grid.best_params_)

{'C': 1000, 'gamma': 0.01, 'kernel': 'rbf'}


# Training

In [ ]:
######################################################################Penalized_SVM (https://www.analyticsvidhya.com/blog/2020/07/10-techniques-to-deal-with-class-imbalance-in-machine-learning/)

# load library
from sklearn.svm import SVC

# we can add class_weight='balanced' to add panalize mistake
svc_model = SVC(kernel='rbf', C= 1000, gamma= 0.01,
                class_weight='balanced', probability=True)

svc_model.fit(Xt_train, Yt_train)


In [ ]:
# Save Model Using joblib
import pandas
from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
import joblib

%cd /content/
# save the model to disk
filename = 'sexualsPenalizedSVM_HPTFinal_UMACCS_model.pkl'
joblib.dump(svc_model, filename)


#Load sexual screening test set

In [ ]:
import pandas as pd
df1= pd.read_csv("sexualsUntrainedTest.csv")
df1

,Unnamed: 0,mol,S_Hit,Smiles
0,25658,<rdkit.Chem.rdchem.Mol object at 0x7fc9363b4c10>,No,Cn1cc(CN2CC[C@@H]3[C@@H](CCC(=O)N3CCc3c[nH]cn3...
1,39559,<rdkit.Chem.rdchem.Mol object at 0x7fc9362189e0>,No,CC1(C)CN=C(N2CCN(CC3CCOC3)CC2)S1
2,36544,<rdkit.Chem.rdchem.Mol object at 0x7fc936195770>,No,COCCC(=O)N1CCN(CC2CC2)c2ccccc21
3,58789,<rdkit.Chem.rdchem.Mol object at 0x7fc935d5bd80>,No,CC(CC(=O)O)CC(=O)Nc1sc2c(c1C(N)=O)CCc1ccccc1-2
4,15213,<rdkit.Chem.rdchem.Mol object at 0x7fc9365e8190>,No,Clc1ccccc1CN1CCc2nc(C3CCCCN3)ncc2C1
...,...,...,...,...
62370,50615,<rdkit.Chem.rdchem.Mol object at 0x7fc935ffa1f0>,No,CC[C@@H](c1ccccc1)n1c(C2CC2)n[nH]c1=O
62371,57063,<rdkit.Chem.rdchem.Mol object at 0x7fc935f145f0>,No,CCn1ncc(S(=O)(=O)N[C@@H]2CC[C@@H](n3ccnc3C)[C@...
62372,19513,<rdkit.Chem.rdchem.Mol object at 0x7fc9364a6ff0>,No,O=C(Nc1ccc(F)cc1)C1CNCCN1C(=O)CC1CCCCC1
62373,41985,<rdkit.Chem.rdchem.Mol object at 0x7fc93607fa00>,No,CNC(=O)CN1CCC(NC(=O)CC(C)CCc2ccccc2)CC1


In [ ]:
compounds = []


for _, compound_key, smiles in df1[["S_Hit", "Smiles"]].itertuples():
    compounds.append((Chem.MolFromSmiles(smiles), compound_key, smiles))


In [ ]:
dff= pd.DataFrame (compounds, columns = ['mol', 'S_Hit', "Smiles"])
sxl=dff
sxl

,mol,S_Hit,Smiles
0,<rdkit.Chem.rdchem.Mol object at 0x7f71c322c9e0>,No,Cn1cc(CN2CC[C@@H]3[C@@H](CCC(=O)N3CCc3c[nH]cn3...
1,<rdkit.Chem.rdchem.Mol object at 0x7f71c322d000>,No,CC1(C)CN=C(N2CCN(CC3CCOC3)CC2)S1
2,<rdkit.Chem.rdchem.Mol object at 0x7f71c322c580>,No,COCCC(=O)N1CCN(CC2CC2)c2ccccc21
3,<rdkit.Chem.rdchem.Mol object at 0x7f71c322c740>,No,CC(CC(=O)O)CC(=O)Nc1sc2c(c1C(N)=O)CCc1ccccc1-2
4,<rdkit.Chem.rdchem.Mol object at 0x7f71c322c7b0>,No,Clc1ccccc1CN1CCc2nc(C3CCCCN3)ncc2C1
...,...,...,...
62370,<rdkit.Chem.rdchem.Mol object at 0x7f71c43d49e0>,No,CC[C@@H](c1ccccc1)n1c(C2CC2)n[nH]c1=O
62371,<rdkit.Chem.rdchem.Mol object at 0x7f71c43d4a50>,No,CCn1ncc(S(=O)(=O)N[C@@H]2CC[C@@H](n3ccnc3C)[C@...
62372,<rdkit.Chem.rdchem.Mol object at 0x7f71c43d4ac0>,No,O=C(Nc1ccc(F)cc1)C1CNCCN1C(=O)CC1CCCCC1
62373,<rdkit.Chem.rdchem.Mol object at 0x7f71c43d4b30>,No,CNC(=O)CN1CCC(NC(=O)CC(C)CCc2ccccc2)CC1


In [ ]:
!pip install deepchem

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:

dpchm= sxl
dpchm
dpchm["Target"] = np.where(dpchm["S_Hit"]== "Yes", 1,0)
dpchm["Target"].value_counts()

dchm= dpchm["mol"]
import deepchem as dc
from deepchem.feat import MACCSKeysFingerprint
#mols = [Chem.MolFromSmiles(x) for x in dchm]
featurizer = MACCSKeysFingerprint()
maccs = featurizer.featurize(dchm)
df=[]
df = pd.DataFrame(maccs)

df

,0,1,2,3,4,5,6,7,8,9,...,157,158,159,160,161,162,163,164,165,166
0,0,0,0,0,0,0,0,0,0,0,...,0,1,0,1,1,1,1,0,1,0
1,0,0,0,0,0,0,0,0,0,0,...,0,1,1,1,1,1,1,1,1,0
2,0,0,0,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,0
3,0,0,0,0,0,0,0,0,0,0,...,0,1,1,1,1,1,1,1,1,0
4,0,0,0,0,0,0,0,0,0,0,...,0,1,0,1,1,1,1,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62370,0,0,0,0,0,0,0,0,0,0,...,0,1,0,1,1,1,0,1,1,0
62371,0,0,0,0,0,0,0,0,0,0,...,0,1,1,1,1,1,1,1,1,0
62372,0,0,0,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,0
62373,0,0,0,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,0


In [ ]:
Xt_test = df
Yt_test= sxl["Target"]

#SVC

In [ ]:
import pandas
from sklearn import model_selection
import joblib

# load the model from disk
svc_model = joblib.load('sexualsPenalizedSVM_HPTFinal_UMACCS_model.pkl')
#result = loaded_model.score(Xt_test, Yt_test)
#print(result)


In [ ]:
######################################################################Penalized_SVM (https://www.analyticsvidhya.com/blog/2020/07/10-techniques-to-deal-with-class-imbalance-in-machine-learning/)

# load library
from sklearn.svm import SVC

svc_predict = svc_model.predict(Xt_test)# check performance


In [ ]:
from sklearn.metrics import recall_score, precision_recall_curve,precision_score, confusion_matrix, accuracy_score, f1_score
# accuracy: (tp + tn) / (p + n)
accuracy = accuracy_score(Yt_test,svc_predict)
print('Accuracy: %f' % accuracy)
recall= recall_score(Yt_test,svc_predict)
print("Recall %f" % recall)
precision= precision_score(Yt_test,svc_predict)
print("precision %f" % precision)
# f1: 2 tp / (2 tp + fp + fn)
f1 = f1_score(Yt_test,svc_predict)
print('F1 score: %f' % f1)
from sklearn.metrics import recall_score, precision_recall_curve,precision_score, confusion_matrix, accuracy_score, f1_score, roc_auc_score
print('ROCAUC score:',roc_auc_score(Yt_test, svc_predict))
print('Accuracy score:',accuracy_score(Yt_test, svc_predict))
print('F1 score:',f1_score(Yt_test, svc_predict))
from sklearn.metrics import recall_score, precision_recall_curve,precision_score, confusion_matrix, accuracy_score, f1_score, roc_auc_score, cohen_kappa_score
# kappa
kappa = cohen_kappa_score(Yt_test, svc_predict)
print('Cohens kappa: %f' % kappa)
# ROC AUC
auc = roc_auc_score(Yt_test, svc_predict)
print('ROC AUC: %f' % auc)
# confusion matrix
matrix = confusion_matrix(Yt_test, svc_predict)
print(matrix)
cm= confusion_matrix(Yt_test,svc_predict)
cm

Accuracy: 0.984080
Recall 0.709402
precision 0.079655
F1 score: 0.143227
ROCAUC score: 0.84699903324819
Accuracy score: 0.9840801603206413
F1 score: 0.14322691975841245
Cohens kappa: 0.140327
ROC AUC: 0.846999
[[61299   959]
 [   34    83]]


array([[61299,   959],
       [   34,    83]])

In [ ]:
print(f"Test Set: {len(Xt_test)}")
print(Yt_test.value_counts())
print(matrix)

Test Set: 62375
0    62258
1      117
Name: Target, dtype: int64
[[61299   959]
 [   34    83]]


# Asexual Models

In [ ]:

import pandas as pd
axl= pd.read_csv("M_asexuals_lib_hits_nd_clst_nhts.csv")
axl['Hit'] = axl['Hit'].replace(np.nan, "No")
axl = axl.iloc[: , 1:]
print(axl['Hit'].value_counts())
axl

In [ ]:
compounds =[]
df1= axl[:20000]

for _, smiles, hit in df1[["Smiles", "Hit"]].itertuples():
    compounds.append((Chem.MolFromSmiles(smiles),hit))

df2= axl[20001:23766]

for _, smiles, hit in df2[["Smiles", "Hit"]].itertuples():
    compounds.append((Chem.MolFromSmiles(smiles),hit))

df3= axl[23767:25396]

for _, smiles, hit in df3[["Smiles", "Hit"]].itertuples():
    compounds.append((Chem.MolFromSmiles(smiles),hit))


df4= axl[25397:25662]

for _, smiles, hit in df4[["Smiles", "Hit"]].itertuples():
    compounds.append((Chem.MolFromSmiles(smiles),hit))

df5= axl[25663:26912]

for _, smiles, hit in df5[["Smiles", "Hit"]].itertuples():
    compounds.append((Chem.MolFromSmiles(smiles),hit))

df6= axl[26913:28399]

for _, smiles, hit in df6[["Smiles", "Hit"]].itertuples():
    compounds.append((Chem.MolFromSmiles(smiles),hit))

df7= axl[28400:]

for _, smiles, hit in df7[["Smiles", "Hit"]].itertuples():
    compounds.append((Chem.MolFromSmiles(smiles),hit))

len(compounds)

In [ ]:
axl["Target"] = np.where(axl["Hit"]== "Yes", 1,0)
axl["Target"].value_counts()

In [ ]:
compounds
dpchm= pd.DataFrame (compounds, columns = ['mol', 'hit'])
dpchm["Target"] = np.where(dpchm["hit"]== "Yes", 1,0)
dpchm["Target"].value_counts()

dchm= dpchm["mol"]
dchm

In [ ]:
!pip install deepchem


In [ ]:
import deepchem as dc
from deepchem.feat import MACCSKeysFingerprint
#mols = [Chem.MolFromSmiles(x) for x in dchm]
featurizer = MACCSKeysFingerprint()
maccs = featurizer.featurize(dchm)
df=[]
df = pd.DataFrame(maccs)
#df["Target"]= sxl["Target"]
df

In [ ]:
dpchm['hit']= np.where(dpchm['hit']== "Yes", 1,0)
dpchm['hit']

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC

Xt_train, Xt_test, Yt_train, Yt_test = train_test_split(df, dpchm['hit'],test_size=0.2, random_state=42)

# Hyperparameter tuning


---



---



In [ ]:
## hyper-parameters (like what C or gamma values to use)
# load library
from sklearn.svm import SVC

# we can add class_weight='balanced' to add panalize mistake
svc_model = SVC( probability=True)

In [ ]:
kernels = ['Polynomial', 'RBF', 'Sigmoid','Linear']#A function which returns the corresponding SVC model
def getClassifier(ktype):
    if ktype == 0:
        # Polynomial kernal
        return SVC(kernel='poly', degree=8, gamma="auto", probability=True)
    elif ktype == 1:
        # Radial Basis Function kernal
        return SVC(kernel='rbf', gamma="auto", probability=True)
    elif ktype == 2:
        # Sigmoid kernal
        return SVC(kernel='sigmoid', gamma="auto", probability=True)
    elif ktype == 3:
        # Linear kernal
        return SVC(kernel='linear', gamma="auto", probability=True)

In [ ]:
## https://www.vebuso.com/2020/03/svm-hyperparameter-tuning-using-gridsearchcv/
import pandas as pd
import numpy as np
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import recall_score, precision_recall_curve,precision_score, confusion_matrix, accuracy_score, f1_score, roc_auc_score
import matplotlib.pyplot as plt
%matplotlib inline

for i in range(4):
    # Separate data into test and training sets
    X_train, X_test, y_train, y_test = train_test_split(df, dpchm['hit'],test_size=0.2, random_state=42)# Train a SVC model using different kernal
    svclassifier = getClassifier(i)
    svclassifier.fit(X_train, y_train)# Make prediction
    y_pred = svclassifier.predict(X_test)# Evaluate our model
    print("Evaluation:", kernels[i], "kernel")
    print(classification_report(y_test,y_pred))

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold

# defining parameter range
#param_grid = {'C': [0.1, 1, 10],
#              'gamma': [1, 0.1, 0.01],
#              'kernel': ['rbf']}

# defining parameter range
param_grid = {'C': [0.1, 1, 10],
              'gamma': [ 0.1, 0.01],
              'kernel': ['rbf']}

grid = GridSearchCV(SVC(class_weight='balanced', probability=True), param_grid, refit = True, verbose = 3)

# fitting the model for grid search
grid.fit(Xt_train, Yt_train)

Fitting 5 folds for each of 6 candidates, totalling 30 fits
[CV 1/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.928 total time=26.4min
[CV 2/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.929 total time=26.2min
[CV 3/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.927 total time=26.9min
[CV 4/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.930 total time=27.9min
[CV 5/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.929 total time=28.3min
[CV 1/5] END .....C=0.1, gamma=0.01, kernel=rbf;, score=0.903 total time=20.8min
[CV 2/5] END .....C=0.1, gamma=0.01, kernel=rbf;, score=0.906 total time=17.4min
[CV 3/5] END .....C=0.1, gamma=0.01, kernel=rbf;, score=0.904 total time=14.7min
[CV 4/5] END .....C=0.1, gamma=0.01, kernel=rbf;, score=0.904 total time=15.1min
[CV 5/5] END .....C=0.1, gamma=0.01, kernel=rbf;, score=0.906 total time=15.2min


In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold

# defining parameter range
#param_grid = {'C': [0.1, 1, 10],
#              'gamma': [1, 0.1, 0.01],
#              'kernel': ['rbf']}

# defining parameter range
param_grid = {'C': [ 1, 10],
              'gamma': [ 0.1, 0.01],
              'kernel': ['rbf']}

grid = GridSearchCV(SVC(class_weight='balanced', probability=True), param_grid, cv=2, refit = True, verbose = 3)

# fitting the model for grid search
grid.fit(Xt_train, Yt_train)

Fitting 2 folds for each of 4 candidates, totalling 8 fits
[CV 1/2] END ........C=1, gamma=0.1, kernel=rbf;, score=0.957 total time=11.4min
[CV 2/2] END ........C=1, gamma=0.1, kernel=rbf;, score=0.956 total time=14.4min
[CV 1/2] END .......C=1, gamma=0.01, kernel=rbf;, score=0.919 total time= 6.3min
[CV 2/2] END .......C=1, gamma=0.01, kernel=rbf;, score=0.919 total time= 6.1min
[CV 1/2] END .......C=10, gamma=0.1, kernel=rbf;, score=0.961 total time=20.8min
[CV 2/2] END .......C=10, gamma=0.1, kernel=rbf;, score=0.958 total time=20.0min
[CV 1/2] END ......C=10, gamma=0.01, kernel=rbf;, score=0.941 total time= 3.7min
[CV 2/2] END ......C=10, gamma=0.01, kernel=rbf;, score=0.940 total time= 3.7min


#Training

In [ ]:
######################################################################Penalized_SVM (https://www.analyticsvidhya.com/blog/2020/07/10-techniques-to-deal-with-class-imbalance-in-machine-learning/)

# load library
from sklearn.svm import SVC

#svm_weights = (clf.coef_ ** 2).sum(axis=0) # coef_ is only available when using a linear kernel

# we can add class_weight='balanced' to add panalize mistake
svc_model = SVC(kernel='rbf', C= 10, gamma= 0.1, probability=True)

svc_model.fit(Xt_train, Yt_train)

In [ ]:
# Save Model Using joblib
import pandas
from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
import joblib

%cd /content/
# save the model to disk
filename = 'asexualsPenalizedSVM_HPTFinal_UMACCS_model.pkl'
joblib.dump(svc_model, filename)


#Load Asexual screening test set

In [ ]:
import pandas as pd
df1= pd.read_csv("asexualsUntrainedTest.csv")
df1

,Unnamed: 0,mol,Hit,Smiles
0,51742,<rdkit.Chem.rdchem.Mol object at 0x7f71149bf680>,No,O=C(NCCCN1CCOCC1)C1CCCN(Cc2ccco2)C1
1,20279,<rdkit.Chem.rdchem.Mol object at 0x7f7114ed6180>,No,Cc1nocc1C(=O)N1CCCC2(CCN(c3cnn(C)c3)C2=O)C1
2,17729,<rdkit.Chem.rdchem.Mol object at 0x7f71219e38b0>,No,Cn1cncc1-c1nnc([C@H]2CCN(Cc3cccnc3)C2)o1
3,28545,<rdkit.Chem.rdchem.Mol object at 0x7f7114dc26c0>,No,CCn1nc(C)c2nc(C(C)C)nc(NCC(C)C)c21
4,24371,<rdkit.Chem.rdchem.Mol object at 0x7f7114f4b140>,No,CN(C)CC1(O)CCCN(Cc2ccccc2OCC(=O)N(C)C)C1
...,...,...,...,...
61024,72080,<rdkit.Chem.rdchem.Mol object at 0x7f7114740cf0>,No,N#Cc1cccc(NC(=O)NC2CCN(CCCCCNC(=O)/C=C/c3ccc(C...
61025,49244,<rdkit.Chem.rdchem.Mol object at 0x7f7114b4a340>,No,NC1(C(=O)NCC2CCCN(Cc3cccc4nonc34)C2)CCC1
61026,47413,<rdkit.Chem.rdchem.Mol object at 0x7f7114afbae0>,No,CNC(=O)C1CCCN(c2nc(-c3ccco3)nc3c2CCNCC3)C1
61027,73164,<rdkit.Chem.rdchem.Mol object at 0x7f711476eb20>,No,CC(Cc1ccc(OC2CCCCC2)cc1)NCC(O)c1cccc(Cl)c1.Cl


In [ ]:
compounds = []


for _, compound_key, smiles in df1[["Hit", "Smiles"]].itertuples():
    compounds.append((Chem.MolFromSmiles(smiles), compound_key, smiles))


In [ ]:
dff= pd.DataFrame (compounds, columns = ['mol', 'Hit', "Smiles"])
axl=dff
axl

,mol,S_Hit,Smiles
0,<rdkit.Chem.rdchem.Mol object at 0x7f7113906500>,No,O=C(NCCCN1CCOCC1)C1CCCN(Cc2ccco2)C1
1,<rdkit.Chem.rdchem.Mol object at 0x7f7113906960>,No,Cc1nocc1C(=O)N1CCCC2(CCN(c3cnn(C)c3)C2=O)C1
2,<rdkit.Chem.rdchem.Mol object at 0x7f7113907680>,No,Cn1cncc1-c1nnc([C@H]2CCN(Cc3cccnc3)C2)o1
3,<rdkit.Chem.rdchem.Mol object at 0x7f71139061f0>,No,CCn1nc(C)c2nc(C(C)C)nc(NCC(C)C)c21
4,<rdkit.Chem.rdchem.Mol object at 0x7f7113906c00>,No,CN(C)CC1(O)CCCN(Cc2ccccc2OCC(=O)N(C)C)C1
...,...,...,...
61024,<rdkit.Chem.rdchem.Mol object at 0x7f7112d2ef80>,No,N#Cc1cccc(NC(=O)NC2CCN(CCCCCNC(=O)/C=C/c3ccc(C...
61025,<rdkit.Chem.rdchem.Mol object at 0x7f7112d2eff0>,No,NC1(C(=O)NCC2CCCN(Cc3cccc4nonc34)C2)CCC1
61026,<rdkit.Chem.rdchem.Mol object at 0x7f7112d2f060>,No,CNC(=O)C1CCCN(c2nc(-c3ccco3)nc3c2CCNCC3)C1
61027,<rdkit.Chem.rdchem.Mol object at 0x7f7112d2f0d0>,No,CC(Cc1ccc(OC2CCCCC2)cc1)NCC(O)c1cccc(Cl)c1.Cl


##MACCSKeysFingerprint

In [ ]:
!pip install deepchem

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
dpchm= axl
#dchm
dpchm["Target"] = np.where(dpchm["Hit"]== "Yes", 1,0)
dpchm["Target"].value_counts()

dchm= dpchm["mol"]
import deepchem as dc
from deepchem.feat import MACCSKeysFingerprint
#mols = [Chem.MolFromSmiles(x) for x in dchm]
featurizer = MACCSKeysFingerprint()
maccs = featurizer.featurize(dchm)
df=[]
df = pd.DataFrame(maccs)
#df["Target"]= sxl["Target"]

df

,0,1,2,3,4,5,6,7,8,9,...,157,158,159,160,161,162,163,164,165,166
0,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,1,1,1,1,1,0
1,0,0,0,0,0,0,0,0,0,0,...,0,1,0,1,1,1,1,1,1,0
2,0,0,0,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,0
3,0,0,0,0,0,0,0,0,0,0,...,0,1,0,1,1,1,1,1,1,0
4,0,0,0,0,0,0,0,0,0,0,...,0,1,1,1,1,1,1,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61024,0,0,0,0,0,0,0,0,0,0,...,1,1,1,0,1,1,1,1,1,0
61025,0,0,0,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,0
61026,0,0,0,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,0
61027,0,0,0,0,0,0,0,0,0,0,...,0,1,0,1,1,1,1,1,1,0


In [ ]:
Xt_test = df
Yt_test= axl["Target"]

# SVC

In [ ]:
import pandas
from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
import joblib

In [ ]:

svc_model = joblib.load('asexualsPenalizedSVM_HPTFinal_UMACCS_model.pkl')

In [ ]:
######################################################################Penalized_SVM (https://www.analyticsvidhya.com/blog/2020/07/10-techniques-to-deal-with-class-imbalance-in-machine-learning/)

# load library
from sklearn.svm import SVC

svc_predict = svc_model.predict(Xt_test)# check performance


In [ ]:
from sklearn.metrics import recall_score, precision_recall_curve,precision_score, confusion_matrix, accuracy_score, f1_score
# accuracy: (tp + tn) / (p + n)
accuracy = accuracy_score(Yt_test,svc_predict)
print('Accuracy: %f' % accuracy)
recall= recall_score(Yt_test,svc_predict)
print("Recall %f" % recall)
precision= precision_score(Yt_test,svc_predict)
print("precision %f" % precision)
# f1: 2 tp / (2 tp + fp + fn)
f1 = f1_score(Yt_test,svc_predict)
print('F1 score: %f' % f1)
from sklearn.metrics import recall_score, precision_recall_curve,precision_score, confusion_matrix, accuracy_score, f1_score, roc_auc_score
print('ROCAUC score:',roc_auc_score(Yt_test, svc_predict))
print('Accuracy score:',accuracy_score(Yt_test, svc_predict))
print('F1 score:',f1_score(Yt_test, svc_predict))
from sklearn.metrics import recall_score, precision_recall_curve,precision_score, confusion_matrix, accuracy_score, f1_score, roc_auc_score, cohen_kappa_score
# kappa
kappa = cohen_kappa_score(Yt_test, svc_predict)
print('Cohens kappa: %f' % kappa)
# ROC AUC
auc = roc_auc_score(Yt_test, svc_predict)
print('ROC AUC: %f' % auc)
# confusion matrix
matrix = confusion_matrix(Yt_test, svc_predict)
print(matrix)
cm= confusion_matrix(Yt_test,svc_predict)
cm

Accuracy: 0.835799
Recall 0.922166
precision 0.207758
F1 score: 0.339115
ROCAUC score: 0.8769157029132671
Accuracy score: 0.835799374068066
F1 score: 0.3391149508672426
Cohens kappa: 0.285864
ROC AUC: 0.876916
[[48437  9804]
 [  217  2571]]


array([[48437,  9804],
       [  217,  2571]])

In [ ]:
print(f"Test Set: {len(Xt_test)}")
print(Yt_test.value_counts())
print(matrix)

Test Set: 61029
0    58241
1     2788
Name: Target, dtype: int64
[[48437  9804]
 [  217  2571]]
